# Mapserver / Mapfile

In [1]:
# local
from AlertaDengue.dados import dbdata

import os
import sqlalchemy as sqla
# local
import AlertaDengue as alert_dengue

In [2]:
def get_passwd_from_ini(filepath: str):
    with open(filepath) as f:
        text = f.read()
        i = text.index('PSQL_PASSWORD')
        f = text[i:].index('\n')
        passwd = text[i:i+f].split('=')
        return passwd[1].strip()
    

## DB connection

In [3]:
db = 'dengue'
user = 'dengueadmin'
host = 'localhost'
passwd = get_passwd_from_ini('../AlertaDengue/AlertaDengue/settings.ini')
dsn = "postgresql://{}:{}@{}/{}".format(
    user, passwd, host, db
)

engine = sqla.create_engine(dsn)

## Mapfile

### Templates

In [59]:
# mapfile templates

mapfile_template = '''
MAP
    IMAGETYPE      png
    EXTENT      -87.93749190049996 -20.493952297946063 -14.892815463099353  7.223000268026496
 
    SIZE           1800 1600
    IMAGECOLOR     0 0 0
    CONFIG "MS_ERRORFILE" "%(ms_error_path)s"
    CONFIG "CPL_DEBUG" "ON"
    CONFIG "PROJ_DEBUG" "ON"
    DEBUG 5
    
    OUTPUTFORMAT
      NAME "GTiff"
      DRIVER GDAL/GTiff
      MIMETYPE "image/tiff"
      IMAGEMODE RGBA
      EXTENSION "tif"
    END
    OUTPUTFORMAT
      NAME "kml"
      DRIVER KML
      MIMETYPE "application/vnd.google-earth.kml.xml"
      IMAGEMODE RGBA
      EXTENSION "kml"
    END

    PROJECTION
      "init=epsg:4326"
    END

    WEB
      METADATA
        "wms_title" "Alerta Dengue"
        "wms_onlineresource" "%(ms_cgi_path)s"
        "wms_enable_request" "*"
        "wms_srs" "EPSG:4326"
        "wms_feature_info_mime_type" "text/html"
        "wms_format" "image/png"
      END
    END
 
    %(layers)s
 
END
'''

mapfile_layer_template = '''
    LAYER
        NAME         "%(city_name)s"
        DATA         "%(shp_path)s/%(geocode)s"
        STATUS       ON
        TYPE         POLYGON
        OFFSITE      0 0 0
        
        PROJECTION
          "init=epsg:4326"
        END
        
        METADATA
          "wms_srs" "EPSG:4326"
        END
        
        COMPOSITE
            OPACITY 70
        END # COMPOSITE
 
        CLASS
            STYLE
                COLOR %(rgb)s
            END
        END
    END
'''

### Setting variables

In [60]:
shp_path = '%s/static/shapefile' % alert_dengue.__path__[0]
ms_error_path = '/home/xmn/alerta_dengue_ms.log'
ms_cgi_path = 'http://localhost:81/cgi-bin/mapserv?map=%s&'
ms_mapfile_name = 'alerta_%s.map'
ms_mapfile_dir = '%s/mapfiles/%%s' % (
    os.path.dirname(alert_dengue.__path__[0])
)

In [6]:
alert_colors = [
    '000 255 000',
    '255 255 000',
    '255 190 000',
    '255 000 000',
]

### Generating the mapfile

In [7]:
sql_template = '''
SELECT geocodigo, nome, uf 
FROM "Dengue_global"."Municipio" 
WHERE uf = '%s'
ORDER BY nome;
'''

In [29]:
state_initials = {
    'Paraná': 'PR', 
    'Rio de Janeiro': 'RJ', 
    'Ceará': 'CE', 
    'Minas Gerais': 'MG', 
    'Espírito Santo': 'ES'
}

In [61]:
layers = {}

for disease in ('dengue', 'chikungunya'):
    layers[disease] = {}
    for state_name in state_initials.keys():
        with engine.connect() as conn:
            sql = sql_template % state_name
            result = conn.execute(sql).fetchall()

        cities_alert = dbdata.NotificationResume.get_cities_alert_by_state(
            state_name, disease
        )

        alerts = dict(
            cities_alert[['municipio_geocodigo', 'level_alert']].values
        )
        
        layers[disease][state_name] = ''

        for geocode, city_name, state_country in result:
            alert_level = alerts[geocode] if geocode in alerts else -1
            # print(alert_level)
            alert_color = (
                alert_colors[alert_level] if 0 <= alert_level <= 3 else
                '200 200 200'  # gray
            )
            layer = mapfile_layer_template % {
                'geocode': geocode,
                'city_name': city_name,
                'rgb': alert_color,
                'shp_path': shp_path
            }
            
            if not os.path.exists('%s/%s.shp' % (shp_path, geocode)):
                layer = layer.replace('\n', '\n# ') + '\n'

            layers[disease][state_name] += layer

for k_disease, states_layer in layers.items():
    layers = ''
    for state_name, layer in states_layer.items():
        # save for individual state
        _ms_mapfile_name = ms_mapfile_name % (
            '%s_%s' % (k_disease, state_initials[state_name])
        )
        _ms_mapfile = ms_mapfile_dir % _ms_mapfile_name
        
        ms_config = {
            'layers': layer,
            'ms_error_path': ms_error_path,
            'ms_cgi_path': ms_cgi_path % _ms_mapfile
        }

        mapfile = mapfile_template % ms_config

        print('Saving ', _ms_mapfile, 'file ...')
        with open(_ms_mapfile, 'w') as f:
            f.write(mapfile)
        # join layers from all states
        layers += layer
    
    _ms_mapfile_name = ms_mapfile_name % k_disease
    _ms_mapfile = ms_mapfile_dir % _ms_mapfile_name
    
    ms_config = {
        'layers': layers,
        'ms_error_path': ms_error_path,
        'ms_cgi_path': ms_cgi_path % _ms_mapfile
    }
    
    mapfile = mapfile_template % ms_config
    
    print('Saving ', _ms_mapfile, 'file ...')
    with open(_ms_mapfile, 'w') as f:
        f.write(mapfile)

print('[II] DONE!')

Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue_PR.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue_RJ.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue_CE.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue_MG.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue_ES.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_dengue.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_chikungunya_PR.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_chikungunya_RJ.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_chikungunya_CE.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_chikungunya_MG.map file ...
Saving  /home/xmn/dev/AlertaDengue/AlertaDengue/mapfiles/alerta_chikungunya_ES.map file ...
Saving  /home/xmn/dev

## Visualizing the map

In [62]:
%%html

<div id="alerta_dengue" style="height: 400px; width: 800px;"></div>

<link rel="stylesheet" href="lib/leaflet/leaflet.css" />
<script type="text/javascript" src="lib/leaflet/leaflet.js"></script>

<script>

var mapfile_url = (
    'http://localhost:81/cgi-bin/mapserv?' + 
    'map=/var/www/mapserver/mapfiles/alerta_dengue.map&' + 
    'mode=map'
);

var map = L.map('alerta_dengue').setView([-14.01,-53.96], 4); 


var osmUrl='http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png';
var osmAttrib = 'Map data © <a href="http://openstreetmap.org">OpenStreetMap</a> contributors';

L.tileLayer(osmUrl, {
    minZoom: 4, 
    maxZoom: 10, 
    attribution: osmAttrib
}).addTo(map);

L.tileLayer.wms(mapfile_url, {
    layers: 'Roads',
    format: 'image/png',
    transparent: true,
    attribution: "Info Dengue",
    maxZoom: 10,
    minZoom: 4,
}).addTo(map);

</script>